In [1]:
# !pip install allennlp

In [2]:
from pathlib import Path
from tqdm import tqdm
import pickle

In [3]:
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch

In [4]:
import pandas as pd

In [5]:
data = Path('data_prep/final_data/en/')
elmo_path = Path('models/elmo_small/')

In [6]:
raw = pd.read_csv('data_prep/data/entries_raw.csv')
train = pd.read_csv(data / 'sentences_en_train.csv')
test = pd.read_csv(data / 'sentences_en_test.csv')

In [7]:
sentences = [x.split() for x in train.sentence_text]

In [8]:
model = Elmo(
    options_file=elmo_path / 'elmo_2x1024_128_2048cnn_1xhighway_options.json',
    weight_file=elmo_path / 'elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5',
    num_output_representations=1
)
model.cuda()

Elmo(
  (_elmo_lstm): _ElmoBiLm(
    (_token_embedder): _ElmoCharacterEncoder(
      (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
      (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
      (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
      (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
      (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
      (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
      (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
      (_highways): Highway(
        (_layers): ModuleList(
          (0): Linear(in_features=2048, out_features=4096, bias=True)
        )
      )
      (_projection): Linear(in_features=2048, out_features=128, bias=True)
    )
    (_elmo_lstm): ElmoLstm(
      (forward_layer_0): LstmCellWithProjection(
        (input_linearity): Linear(in_features=128, out_features=4096, bias=False)
        (state_linearity): Linear(in_features=128, out_featu

In [9]:
output = []
batch_size = 64
with torch.no_grad():
    for i in tqdm(range(0, len(sentences), batch_size*10)):
        batch = batch_to_ids(sentences[i:i+batch_size])[:, :350, :].cuda()
#         print(batch.shape)
    #     print(batch.shape)
        output.append(model(batch)['elmo_representations'][0].cpu())
#         batch.cpu()

100%|██████████| 300/300 [05:19<00:00,  1.06s/it]


In [10]:
len(output)

3000

In [11]:
# base = Path('/home/ec2-user/SageMaker/experiments-dfs/models/elmo_small/results/sentence_embedding')
# for i in tqdm(range(0, len(output), 64)):
#     with open(base / f'{i}.pickle', 'wb') as f:
#         pickle.dump(output[i:i+64], f)

In [12]:
output[0].shape

torch.Size([64, 84, 256])

In [13]:
sentence_output = [x.mean(axis=1) for x in output]
sentence_output = torch.cat(sentence_output)

In [14]:
with open(base / f'sentence_output.pickle', 'wb') as f:
    pickle.dump(sentence_output, f)

NameError: name 'base' is not defined

In [ ]:
!pwd